In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import cv2
import os
from tensorflow.keras.utils import load_img,img_to_array

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
import cv2
import numpy as np

np.random.seed(1234)

path = '/content/drive/MyDrive/ECG_IMAGES_DATASET'
img_list = os.listdir(path)

label_dict = {
    'ECG Images of Myocardial Infarction Patients (240x12=2880)': 0,
    'ECG Images of Patient that have History of MI (172x12=2064)': 1,
    'ECG Images of Patient that have abnormal heartbeat (233x12=2796)': 2,
    'Normal Person ECG Images (284x12=3408)': 3,
    'Another Set of Normal Person ECG Images (284x12=3408)': 4  # Corrected label for the duplicate entry
}

data = []
label = []

for cat in img_list:
    pic_list = os.path.join(path, cat)

    # Check if the item in the directory is a subdirectory
    if os.path.isdir(pic_list):
        for img in os.listdir(pic_list):
            image_path = os.path.join(pic_list, img)

            # Check if the item is a file
            if os.path.isfile(image_path):
                image = cv2.imread(image_path)
                image = image[300:1480, 150:2125]
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, (224, 224))
                data.append(image)
                label.append(label_dict.get(cat, -1))  # Use get() to handle missing keys gracefully

print("Total images processed:", len(data))


Total images processed: 1178


In [6]:
data=np.array(data)
data.shape

(1178, 224, 224, 3)

In [7]:
label=np.array(label)
label.shape

(1178,)

In [5]:
import keras
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Flatten,Conv2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0,EfficientNetB3

In [8]:
num_classes = 5
label = keras.utils.to_categorical(label, num_classes)

In [9]:
from sklearn.utils import shuffle

data, label = shuffle(data, label, random_state=42)

In [10]:
model = EfficientNetB0(include_top=True, weights=None, input_shape=(224, 224, 3), classes=5)

In [11]:
model.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 224, 224, 3)          0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 224, 224, 3)          7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 stem_conv_pad (ZeroPadding  (None, 225, 225, 3)          0         ['normalization[0

In [12]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.0001),metrics=['accuracy'])

In [13]:
history=model.fit(data,label,batch_size=16,epochs=20, validation_split=0.1)

Epoch 1/20
67/67 [==============================] - 365s 5s/step - loss: 2.1755 - accuracy: 0.4019 - val_loss: 1.6584 - val_accuracy: 0.2373
Epoch 2/20
67/67 [==============================] - 330s 5s/step - loss: 1.1235 - accuracy: 0.5991 - val_loss: 2.3626 - val_accuracy: 0.2373
Epoch 3/20
67/67 [==============================] - 332s 5s/step - loss: 1.0211 - accuracy: 0.6981 - val_loss: 2.4828 - val_accuracy: 0.2373
Epoch 4/20
67/67 [==============================] - 327s 5s/step - loss: 0.7637 - accuracy: 0.7377 - val_loss: 2.2137 - val_accuracy: 0.2373
Epoch 5/20
67/67 [==============================] - 326s 5s/step - loss: 0.6688 - accuracy: 0.7858 - val_loss: 4.2839 - val_accuracy: 0.2373
Epoch 6/20
67/67 [==============================] - 327s 5s/step - loss: 0.5573 - accuracy: 0.8340 - val_loss: 6.4511 - val_accuracy: 0.2373
Epoch 7/20
67/67 [==============================] - 327s 5s/step - loss: 0.5635 - accuracy: 0.8038 - val_loss: 3.5819 - val_accuracy: 0.2373
Epoch 8/20
67

In [14]:
model.save("updated_ECG_ECN.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
! pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00


In [19]:
!wget -q -O - ipv4.icanhazip.com

34.90.0.163


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501

[..................] \ fetchMetadata: sill resolveWithNewModule debug@4.3.2 che


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.90.0.163:8501

npx: installed 22 in 2.361s
your url is: https://hungry-tools-shake.loca.lt
2023-12-01 16:13:32.710091: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-01 16:13:32.710167: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-01 16:13:32.710203: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-01 16:13:34.031745: W tensorflow/compiler/tf2tensorrt/utils/p

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import cv2
import os
from tensorflow.keras.utils import load_img,img_to_array